In [1]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
#unsloth/gemma-2b-it-bnb-4bit
#SerchiBoi/DTT-Chatbot-Piloto
#SerchiBoi/DTT-Chatbot-Piloto-v1

#DTT-unsloth-gemma-2b-it-v1
#DTT-unsloth-gemma-2b-it-v2
#DTT-unsloth-gemma-2b-it-v3
#DTT-unsloth-gemma-2b-it-v4
#DTT-unsloth-gemma-2b-it-v5

In [3]:
#@title Import model

In [4]:
model = "SerchiBoi/DTT-Chatbot-Piloto-v1" 

In [5]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [6]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 3050. Max memory: 7.779 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.5 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [7]:
#@title Gmail Functionality

In [8]:
# Define the function to be called when the button is clicked
from datetime import datetime, timedelta
momento_inicial = None
def on_button_click():
    global momento_inicial
    if momento_inicial is None:
        momento_inicial = datetime.now()
    else:
        diferencia = datetime.now() - momento_inicial
        intervalo_deseado = timedelta(minutes=5)
        if diferencia >= intervalo_deseado:
            momento_inicial = None
            return on_button_click()
        else:
            tiempo_restante = intervalo_deseado - (datetime.now() - momento_inicial)
            tiempo_restante_str = str(tiempo_restante).split('.')[0]
            return f"Ya se tiene en cola un reporte hace menos de 5 minutos. Podrás enviar uno en {tiempo_restante_str}."
    sendMils()
    return "¡Aviso enviado, gracias por el reporte!"

In [9]:
import smtplib
def sendMils():
    me =  "dttchatbot@gmail.com"
    family = [
        "sergieari@gmail.com",
        "3000523830101@ingenieria.usac.edu.gt",
        "fedeloboggonzales@gmail.com",
        me
    ]
    
    subject = "Aviso de panico"
    message = "Se ha recibido una alerta de panico."
    
    text = f"Subject: {subject}\n\n{message}"
    
    server = smtplib.SMTP("smtp.gmail.com",587)
    server.starttls()
    
    server.login(me,"pldhxmaifoajsiyr")
    
    for reciber in family:
        server.sendmail(me,reciber,text)

In [10]:
#@title Traslate Functionality

In [11]:
# Funtion to translate from Spanish to English and vice versa
from deep_translator import GoogleTranslator

def translate_text(text,mode):
        try:
            # print("Traduciendo:",text,"--",type(text))
            # Define the source and destination of the translation
            src = 'es' if mode else 'en'
            dest = 'en' if mode else 'es'
            #print(src,"--",dest)

            # Call the Google API to do the translation
            translated_text = GoogleTranslator(source=src, target=dest).translate(text) 
            # print("Resultado:",translated_text,type(translated_text))
            
            return translated_text
        except Exception as e:
            print(f"Error al traducir:{text} --->{e}")
            return "Tuve un problema de traducción ¿puedes volver a preguntar?"  # In case of error, returns the original text

In [12]:
#@title Chat Functionality

In [13]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer,TextIteratorStreamer
from threading import Thread

def chat_gemma_2b(message: str, history: list  ) -> str:
    message = translate_text(message, True)  # Translate es to en

    #for user, assistant in history:
    #    conversation.extend([
    #                         {"role": "user", "content": translate_text(user,True)}, # Add the user message
    #                         #{"role": "user", "content": user}, # Add the user message
    #                         {"role": "assistant", "content": translate_text(assistant,True)}# Add the assistant response
    #                         #{"role": "assistant", "content": assistant}# Add the assistant response
    #                        ])
    
    messages = [
        {"role": "user", "content": message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, 
        tokenize = True, 
        add_generation_prompt = True, 
        return_tensors="pt"
    ).to(model.device)
    
    text_streamer = TextIteratorStreamer(
        tokenizer, 
        timeout=10.0,
        skip_prompt=True, 
        skip_special_tokens=True
    )

    generate_kwargs = dict(
        input_ids= input_ids,
        streamer=text_streamer,
        max_new_tokens=2048,
        use_cache=True
        #do_sample=True,
    )

    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    response = ""
    for new_text in text_streamer:
        response += new_text
        yield response
    response = translate_text(response,False)
    response = response.replace("TDT", "DTT")
    yield response
 

In [14]:
#@title Gradio Functionality

In [15]:
DESCRIPTION = '''
<div>
<h1 style="text-align: center;">DTT Chatbot Piloto</h1>
<h3></h3>
<h2>🦕 Este chatbot piloto está diseñado para responder preguntas relacionadas con las prácticas de la ECYS, centrándose especialmente en las prácticas finales vinculadas al proyecto DTT.</h2>
<h3></h3>
<h2>✅ Recuerda completar el <a href="https://forms.gle/7a6A3mx3fTj8Xo3aA"><b>fomulario con tu experiencia</b></a>, para su posterior análisis.</h2>
<h3></h3>
<h3>⚠️ Para obtener respuestas precisas del chatbot, es esencial escribir preguntas correctamente y usar signos de puntuación adecuadamente. Esto ayuda a la comprensión y a obtener respuestas rápidas y precisas. </h3>
<h3></h3>
<h3></h3>
</div>
'''

PANIC = '''
<div>
<h1> </h1>
<h1>❗¿El chatbot está tardando mucho en responder o no responde?❗ </h1>
<h3>¡Presiona el botón de "<b>Aviso de pánico</b>" para enviar un aviso y que el chatbot sea reseteado manualmente!</h3>
</div>
'''

FOOTER = '''
<div>
<h1> </h1>
<h1>🚏 Especificaciones del chatbot 🚏 </h1>
<h2>💡 Aquí puedes encontrar el <a href="https://huggingface.co/datasets/SerchiBoi/DTT-Info"><b>conjunto de datos de entrenamiento</b></a></h2>
<h2>📢 Aquí puedes encontrar el <a href="https://huggingface.co/SerchiBoi/DTT-Chatbot-Piloto-v1"><b>modelo actualmente usado</b></a></h2>
</div>
'''

LICENSE = """
<p/>
---
Built with Gemma 2B-it
"""

PLACEHOLDER = """
<div style="padding: 30px; text-align: center; display: flex; flex-direction: column; align-items: center;">
   <img src="https://www.lavanguardia.com/andro4all/hero/2024/04/google-gemma.png?width=1200&aspect_ratio=16:9" style="width: 80%; max-width: 550px; height: auto; opacity: 0.55;"> 
   <h1 style="font-size: 28px; margin-bottom: 2px; opacity: 0.55;">Gemma 2B-it</h1>
   <p style="font-size: 18px; margin-bottom: 2px; opacity: 0.65;">Pregúntame cualquier duda...</p>
</div>
"""


css = """
h1 {
  text-align: center;
  display: block;
}
#duplicate-button {
  margin: auto;
  color: white;
  background: #1565c0;
  border-radius: 100vh;
}
"""

In [16]:
import gradio as gr

# Gradio block
chatbot=gr.Chatbot(height=450, placeholder=PLACEHOLDER, label='Gradio ChatInterface')

with gr.Blocks(fill_height=True, css=css) as demo:
    """
    Create a Gradio interface for the GEMMA 2B chat model.

    Parameters:
    fill_height (bool): Specifies if the height of the interface should be filled.
    css (str): Custom CSS for styling the interface.
    """
    gr.Markdown(DESCRIPTION) # Add a Markdown section with the description of the interface
    
    # Create a chat interface for interacting with the GEMMA 2B model
    gr.ChatInterface(
        fn=chat_gemma_2b, # Function to call for generating responses
        chatbot=chatbot,  # Chatbot object to handle the conversation
        fill_height=True, # Specifies if the height of the chat interface should be filled
        additional_inputs_accordion=gr.Accordion(label="⚙️ Parameters", open=False, render=False), # Accordion for additional input parameters
        examples=[

            ['¿Qué es la Universidad de San Carlos de Guatemala?'],
            ['¿Qué es la Escuela de Ciencias y Sistemas de la Universidad de San Carlos de Guatemala?'],
            ['¿Cuál es el objetivo principal del proyecto “Desarrollo de Transferencia Tecnológica (DTT) en la Escuela de Ingeniería en Ciencias y Sistemas de la Facultad de Ingeniería, USAC?'],
            ['¿Cómo se dividen las prácticas de ingeniería?|'],
            ['¿Cuánto tiempo dura la práctica inicial?'],
            ['¿Cuánto tiempo dura la práctica intermedia?'],
            ['¿Cuánto tiempo dura la práctica final laboral?'],
            ['¿Cuánto tiempo dura la práctica de empresarios juveniles?'],
            ['¿Qué se considera una falta leve durante las Prácticas Finales?'],
            ['¿Qué se considera una falta grave durante las Prácticas Finales?'],
            ['¿Cuáles son las competencias que debe tener un egresado de la Licenciatura en Ingeniería en Ciencias y Sistemas?'],
            ['¿Qué correo se debe utilizar en el proceso de prácticas docentes dentro del proyecto DTT?'],
            ['¿Como creo la ponderación de un curso en DTT?'],
            ['¿Cuál es el propósito de los laboratorios de la Escuela de Ciencias y Sistemas?'],
            ['¿Dónde puedo encontrar información sobre el Acuerdo de Cumplimiento Ético y Profesional de la Práctica Docente?'],
            

            ['¿Donde puedo encontrar el documento Manual de comportamiento?'],
            ['¿Donde puedo encontrar la plantilla de actividades?'],
            ['¿Donde puedo encontrar la plantilla de inicio?'],
            ['¿Donde puedo encontrar el documento Manual DSI-104?'],
            ['¿Donde puedo encontrar la plantilla de foros?'],
            ['¿Donde puedo el link de entrega de foros?'],
            ['¿Donde puedo el link del documento de control de asistencia?'],
            ['¿Donde puedo el link de entrega de grabaciones?'],
            ['¿Donde puedo el link de entrega de hojas de calificación?'],
            ['¿Donde puedo encontrar el documento Perfil de Egreso del Estudiante de Ingeniería en Ciencias y Sistemas?'],
            ['¿Donde puedo encontrar el documento Procedimiento de Conferencias Online?'],
            ['¿Donde puedo encontrar el documento Proyecto DTT?'],
            ['¿Donde puedo encontrar el ejemplo de como crear la ponderación en DTT?'],
            ['¿Donde puedo encontrar el ejemplo del formato del reporte de conferencia?'],
            ['¿Donde puedo encontrar el ejemplo del reporte mensual DTT?'],
            ['¿Donde puedo encontrar el documento de Acuerdo Anti-Plagio?'],
            ['¿Donde puedo encontrar la plantilla Formato Reporte de conferencia?'],
            ['¿Donde puedo encontrar la plantilla de SLA de Cumplimiento Etico y Profesional de Practica Proyecto DTT?'],
            ['¿Donde puedo encontrar la plantilla de Hoja de Calificación?'],
            ['¿Donde puedo encontrar guía para la elaboración de un artículo?']
            ],
        cache_examples=False, # Do not cache the examples
        examples_per_page=15
    )
    gr.Markdown(PANIC) # Add a Markdown section with the description of the interface
    # Add a custom button and connect it to the on_button_click function
    custom_button = gr.Button("Aviso de pánico")
    output_text = gr.Textbox(label="Resultado del aviso:")
    custom_button.click(on_button_click, inputs=[], outputs=output_text)

    gr.Markdown(FOOTER)

    # Add a Markdown section with the license information
    gr.Markdown(LICENSE)

In [17]:
 # demo.launch()

In [18]:
#demo.launch(share=True)
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
